In [ ]:
import pandas as pd
import numpy as np
from utils import *


In [ ]:
#pip install sentence-transformers

Load the datasets - Mentor/Mentee/Circle

In [ ]:
Mentee_df = pd.read_excel('../data/Mentee-obfuscated-dataset-1.xlsx', engine='openpyxl')

In [ ]:
Mentee_df.columns

In [ ]:
Mentor_df = pd.read_excel('../data/Mentor-obfuscated-dataset-1.xlsx', engine='openpyxl')

In [ ]:
Mentor_df.columns

In [ ]:
Circle_df = pd.read_excel('../data/CircleResult.xlsx', engine='openpyxl')

In [ ]:
#Circle_df.head()

In [ ]:
list(Circle_df.columns)

Select only 3 columns

In [ ]:
Circle_basic_df = Circle_df[['Circle','Mentor_ID','Mentee_ID']]

In [ ]:
#Circle_basic_df.head()
n = Circle_df.Mentee_Email.nunique()
  
print("No.of.unique values ",n)

In [ ]:
Cicle_basic_sorted_df = Circle_basic_df.sort_values(by=['Circle','Mentor_ID','Mentee_ID'], inplace=False, ignore_index= True)

For each circle row, get the mentor and mentee information and create a detail dataframe

• Organization/Department - Mentor and Mentee should not be in same Department. 
• Time Zones – Fit circles into similar time zones. Find difference of timezone between mentor and mentee
• Grade Levels/Mentor Circle Levels – Anyone Grade level less than 28 should not be considered a mentor.
• CLL Sort – Mentors strengths match the Mentees growth areas.
• Areas of interest – Maximum match on areas of interest within a circle.
• Circle size –The number of mentees a mentor has. The circles should be close in size.
• Mentor grade error if mentor grade < 28




In [ ]:
Result_Dictionary={}
Result_DF = pd.DataFrame()

In [ ]:
Cicle_basic_sorted_dfdict = Cicle_basic_sorted_df.to_dict('records')
for  circle_row in Cicle_basic_sorted_dfdict:
    mentorId = circle_row['Mentor_ID']
    menteeId = circle_row['Mentee_ID']
    if (np.isnan(mentorId) or mentorId ==' ' or np.isnan(menteeId) or menteeId ==' '):
        continue
    else:        
        mentor_row = Mentor_df[Mentor_df['Mentor_ID'] ==circle_row['Mentor_ID'] ]
        mentee_row = Mentee_df[Mentee_df['Mentee_ID'] ==circle_row['Mentee_ID'] ]

        print(circle_row)
        result = validate_mentormentee(mentor_row, mentee_row, circle_row)
        result['Mentor_Email'] = mentor_row['Mentor_Email'].iloc[0]
        result['Mentee_Email'] = mentee_row['Mentee_Email'].iloc[0]
        Result_DF = Result_DF.append(result, ignore_index=True)
    

In [ ]:
Result_DF.head()

Create a summary dataframe  
• Count of members in cicle
• Min Max on interests, CLL sort
• Count of departments in cicle

In [ ]:

# Groupby multiple columns & multiple aggregations
grouped = Result_DF.groupby('Circle').aggregate({'Mentor_ID':['count','nunique'],'Mentee_ID':['count','nunique'],'Mentee_TimeZoneNum':['min','max'], 'Mentor_Eligible_Error':'sum', 'MentorMentee_Grade_Error':'sum','MentorMentee_DeptError':'sum','Mentee_Department':'nunique','Goal_Similarity':['min','max'],'Mentee_Grade':['min','max'],'Mentee_TimeZoneNum':['min','max'],'Mentor_Mentee_TimeZoneDist':['min','max'],'MentorMentee_CLL_Growth':['min','max'],'MentorMentee_Circle_Error':['sum'],'MentorMentee_Common_Interests':['min','max']})
regroup_DF = grouped.reset_index()
regroup_DF.head()

Calculate Circle level Penalty and Bonus

• Organization/Department - To encourage networking the circles mentees from the same organization should be split between different circles. Mentor and Mentee should not be in same Department.
• Time Zones – Fit circles into similar time zones or within 2-hour differences. *International participants placed in eastern time zone groups.
• Grade Levels/Mentor Circle Levels – Should not have more than three grade levels between participants. Appropriate grade level grouping of mentor and mentees. Circles should not have both Early Career and Mid Career members.
Anyone Grade level less than 28 should not be considered a mentor.
• CLL Sort – Mentors strengths match the Mentees growth areas.
• Areas of interest – Maximum match on areas of interest within a circle.
• Circle size –The number of mentees a mentor has. The circles should be close in size.


In [ ]:
#Set Base Score as 100
Base_Score = 100

 Create Circle summary score and explanation`

In [ ]:

Mentee_Department_In_Circle_Error_Group = Result_DF.groupby(['Circle', 'Mentee_Department']).filter(lambda x: len(x) >= 2).Circle.unique()


In [ ]:
Circle_Penalty, Circle_Bonus,  Circle_Summary_DF = calculateCircleLevelScore(regroup_DF, Mentee_Department_In_Circle_Error_Group)


In [ ]:
Circle_Summary_DF

In [ ]:
Circle_Summary_DF.columns

In [ ]:
print("Circle score is ",Circle_Bonus, Circle_Penalty)

Create high level statistics for easier evaluation of bonus and penalty points

Penalty points:
    Circle size > 10
    Mentor and mentee are in same department
    Number of mentees not matched to any mentor
Bonus Points
    Timezone 
    Grade Level distance <= 3
    Drop mentors with grade level < 28
      
      
• Max - Min circle size
• Circle Size > 10
• Number of mentees not in any circle
• Number of mentors not in any circle
• Duplicate entry for mentee

In [ ]:
#Get High Level Score 
(Summary_Score, Summary_Results) = createSummary(Result_DF, regroup_DF)


In [ ]:
##Find unassigned mentors, mentees
unassignedMentees = set(Mentee_df.Mentee_Email).difference(set(Result_DF.Mentee_Email))
unassignedMentors = set(Mentor_df.Mentor_Email).difference(set(Result_DF.Mentor_Email))
Summary_Results['Unassigned Mentees']=";".join(unassignedMentees)
Summary_Results['Number of Unassigned Mentees']=len(unassignedMentees)
Summary_Results['Unassigned Mentors']=";".join(unassignedMentors)
Summary_Results['Number of Unassigned Mentors']=len(unassignedMentors)

In [ ]:
#Add scores to Summary Results
Final_Score = Base_Score + Circle_Bonus + Circle_Penalty + Summary_Score
Summary_Results['Base Score'] = Base_Score
Summary_Results['Circle Level Bonus'] = Circle_Bonus
Summary_Results['Circle Level Penalty'] = Circle_Penalty
Summary_Results['Summary Level Score'] = Summary_Score

Summary_Results['Final Score'] = Final_Score

In [ ]:
Summary_Results_DF = pd.DataFrame.from_dict(data=Summary_Results, orient='index', columns=['Result'])



https://towardsdatascience.com/5-pandas-group-by-tricks-you-should-know-in-python-f53246c92c94

In [ ]:
#Summary_Results_DF.columns

In [ ]:
Summary_Results_DF.index = Summary_Results_DF.index.set_names('Category')
 
Summary_Results_DF.reset_index().head()

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_multiple.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
Summary_Results_DF.to_excel(writer, sheet_name='highlevelsummary', header='False')
Circle_Summary_DF.to_excel(writer, sheet_name='circlesummary')
Result_DF.to_excel(writer, sheet_name='detail')

writer.close()

Group by Circle and Deprtment to check if not more than 2 mentees of same department are not in same circle